# Initialization

In [ ]:
# %load init.ipy
%reload_ext autoreload
%autoreload 2

import os, sys

import numpy as np
import scipy as sp
import scipy.integrate
import matplotlib.pyplot as plt
import matplotlib as mpl

CWD = os.path.abspath(os.path.curdir)
print("CWD: '{}'".format(CWD))

ODIR = os.path.join(CWD, "output", "")
if not os.path.exists(ODIR):
    os.makedirs(ODIR)
    print("Created output directory: '{}'".format(ODIR))

par_dir = os.path.join(CWD, os.path.pardir)
if par_dir not in sys.path:
    sys.path.append(par_dir)
    print("Added parent directory: '{}'".format(par_dir))

import bhem
import bhem.basics
import bhem.utils
import bhem.disks
import bhem.radiation
import bhem.spectra
from bhem.constants import MSOL, H_PLNK, K_BLTZ, SPLC, MPRT, MELC, QELC

np.seterr(over='ignore');

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times']})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

FS_TITLE = 20
FS_LABEL = 16

plt.rcParams.update({'axes.titlesize': FS_TITLE})
plt.rcParams.update({'axes.labelsize': FS_LABEL})
plt.rcParams.update({'xtick.labelsize': FS_LABEL})
plt.rcParams.update({'ytick.labelsize': FS_LABEL})


In [ ]:
freqs = np.logspace(5, 25, 100)

# Thin vs. ADAF

In [ ]:
_MASS = 1e9 * MSOL

fig, ax = plt.subplots(figsize=[8, 5])
ax.set(xscale='log', xlabel='Frequency [Hz]', xlim=[1e5, 1e22],
       yscale='log', ylabel='$\\nu \, F_\\nu [\mathrm{erg \, s}^{-1}]$')
ax.grid(True, which='major', axis='both', c='0.5', alpha=0.5)

# tw = ax.twiny()

fedds = np.logspace(-6, 0, 7)[::-1]

cmap = mpl.cm.get_cmap('gist_heat_r')
colors = [cmap(xx) for xx in np.linspace(0.1, 0.9, fedds.size)]
ymax = 0.0

for ii, fe in enumerate(fedds):
    label = '${:+.1f}$'.format(np.log10(fe))
    cc = colors[ii]
    kw = dict(color=cc, lw=2.0, alpha=0.8, label=label)

    if fe < 1e-2:
        _adaf = bhem.disks.ADAF(_MASS, 100, fedd=fe)
        _spec = bhem.spectra.Mahadevan96(_adaf, freqs)
        lum = freqs*_spec.spectrum
        
        ax.plot(freqs, lum, **kw)
    else:
        _thin = bhem.disks.Thin(_MASS, 100, fedd=fe)
        bb_lum = _thin.blackbody_spectral_luminosity(freqs)
        lum = freqs*bb_lum
        ax.plot(freqs, lum, **kw)
    
    ymax = np.maximum(lum.max(), ymax)
    
ax.set_ylim([1e30, 3*ymax])
ax.text(0.98, 0.98, "$M = {:.1e} \,\, M_\odot$".format(_MASS/MSOL), transform=ax.transAxes,
         ha='right', va='top')
    
for nn, band in bhem.constants.BANDS.items():
    ax.axvline(band.freq, color=band.color, lw=1.0, alpha=0.5)

ax.legend(title="$\log(\dot{M}/\dot{M}_\mathrm{edd})$", fontsize=12)
plt.show()

fname = 'spectra_thin-adaf_mdot_2'
fname = os.path.join(ODIR, fname)
fig.savefig(fname + '.pdf')
fig.savefig(fname + '.png')
print("Saved to '{}'".format(fname))

In [ ]:
_MASS = 1e9 * MSOL

fig, axes = plt.subplots(figsize=[12, 5], ncols=2)
plt.subplots_adjust(wspace=0.55, left=0.08, right=0.92, top=0.96)

for ax in axes:
    ax.set(xscale='log', yscale='log')
    ax.grid(True, which='major', axis='both', c='0.5', alpha=0.5)


ax = axes[0]
ax.set(xlim=[1e5, 1e22], xlabel='Frequency [Hz]', 
       ylabel='$\\nu \, F_\\nu [\mathrm{erg \,\, s}^{-1}]$')
tw = ax.twinx(); tw.set(yscale='log', ylabel='Cumulative Luminosity $[\mathrm{erg \,\, s}^{-1}]$')

fedds = np.logspace(-6, 0, 7)[::-1]
lums = np.zeros_like(fedds)

cmap = mpl.cm.get_cmap('gist_heat_r')
colors = [cmap(xx) for xx in np.linspace(0.1, 0.9, fedds.size)]
ymax = 0.0

for ii, fe in enumerate(fedds):
    label = '${:+.1f}$'.format(np.log10(fe))
    cc = colors[ii]
    kw = dict(color=cc, lw=2.0, label=label)

    if fe < 1e-2:
        _adaf = bhem.disks.ADAF(_MASS, 100, fedd=fe)
        _spec = bhem.spectra.Mahadevan96(_adaf, freqs)
        lum = _spec.spectrum
    else:
        _thin = bhem.disks.Thin(_MASS, 100, fedd=fe)
        bb_lum = _thin.blackbody_spectral_luminosity(freqs)
        lum = bb_lum
    
    ax.plot(freqs, freqs*lum, ls='--', alpha=0.5, **kw)
    ymax = np.maximum(np.max(freqs*lum), ymax)
    
    lum_mid = bhem.utils.log_midpoints(lum)
    freqs_mid = bhem.utils.log_midpoints(freqs)
    df = np.diff(freqs)
    cumlum = np.cumsum(df * lum_mid)
    lums[ii] = cumlum[-1]
    tw.plot(freqs_mid, cumlum, alpha=0.8, **kw)    
    
tw.set_ylim([1e32, 1e50])
ax.set_ylim([1e30, 3*ymax])
ax.text(0.02, 0.98, "$M = {:.1e} \,\, M_\odot$".format(_MASS/MSOL), transform=ax.transAxes,
         ha='left', va='top')
    
for nn, band in bhem.constants.BANDS.items():
    ax.axvline(band.freq, color=band.color, lw=1.0, alpha=0.5)

ax.legend(title="$\log(\dot{M}/\dot{M}_\mathrm{edd})$", fontsize=12, loc='center left')


ax = axes[1]
ax.set(xlabel='Eddington Fraction', 
       ylabel='$L_\mathrm{bol} [\mathrm{erg \,\, s}^{-1}]$')
tw = ax.twinx(); tw.set(yscale='log', ylabel='Efficiency')

mdot_edd = bhem.basics.eddington_accretion(_MASS)
effs = lums/(mdot_edd * fedds * SPLC**2)

ax.plot(fedds, lums, 'r-', alpha=0.8)
tw.plot(fedds, effs, 'r--', alpha=0.8)
tw.plot(fedds, np.minimum(10*fedds, 0.1), color='0.5', ls='--', alpha=0.5)

plt.show()

fname = 'lum-eff_thin-adaf_mdot'
fname = os.path.join(ODIR, fname)
fig.savefig(fname + '.pdf')
fig.savefig(fname + '.png')
print("Saved to '{}'".format(fname))